In [4]:
#pandasData = pd.DataFrame(data)
#pandasData.memory_usage
import pickle as pkl
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


batch_size = 1
# load data
data = pd.read_pickle(".\\RML2016.10a\\RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels
qpsk_2_data_all[0]

array([[-5.90147125e-03, -2.34581786e-03, -7.45061261e-04,
        -5.34572452e-03, -5.78941777e-03, -3.69683490e-03,
        -4.97868750e-03, -6.56572822e-03, -9.04932246e-03,
        -4.83668642e-03, -1.00837136e-02, -4.53815702e-03,
        -4.31498839e-03, -5.13423281e-03, -6.07567281e-03,
         1.18665886e-03, -4.65670088e-03, -6.95332745e-03,
        -6.66823424e-03, -6.43977243e-03, -3.82532272e-03,
        -8.38821847e-03, -1.01344110e-02, -6.90073194e-03,
        -9.62839276e-03, -1.55354582e-03, -2.88469438e-03,
        -4.51788818e-03,  3.41027649e-03,  7.41052255e-03,
         3.35769332e-03,  7.62627879e-03,  8.82679410e-03,
         3.42824613e-03,  1.84083998e-03,  6.41621463e-03,
        -1.63305740e-04, -2.24135863e-03, -5.19226259e-03,
        -3.63920978e-03, -1.01316329e-02, -6.39987178e-03,
        -6.06458448e-03, -7.66557641e-03, -3.44835571e-03,
         4.42530581e-04,  2.56719789e-03,  4.74519981e-03,
         4.66336496e-03,  6.47741836e-03,  8.53952859e-0

In [5]:
# convert labels to NumPy array and then to PyTorch tensor with Long data type bc torch.nn.CrossEntropyLoss requires long Datatype
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = torch.from_numpy(labels_combined).long()

# convert 2 PyTorch tensor
data_combined = torch.from_numpy(data_combined).float()

# convert labels 2 NumPy array and then 2 PyTorch tensor
labels_combined = np.array(labels_combined)
labels_combined = torch.from_numpy(labels_combined)

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # access a single data sample and label
        sample = self.data[idx]
        label = self.labels[idx]
        #print(f'raw sample: {sample}')
        # Convert sample, min_vals, and max_vals to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        #print(f'raw sample: {sample}')
        min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
        
        
        #print(f'min val {min_vals}')

    
        max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
        #print(f'max val {max_vals}')
        #normalize
        epsilon = 1e-10
        normalized_sample = 2 * (sample - min_vals.unsqueeze(1)) / (max_vals.unsqueeze(1) - min_vals.unsqueeze(1) + epsilon) - 1
        #print(f'Normalized sample: {normalized_sample}')
        return normalized_sample, label

#train_dataset = MyDataset(data_train, labels_train)
#test_dataset = MyDataset(data_test, labels_test)

#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
#test_loader = DataLoader(test_dataset, batch_size=batch_size)
single_dataset_bpsk = MyDataset(bpsk_2_data_all, bpsk_labels)
single_loader_bpsk = DataLoader(single_dataset_bpsk, batch_size=batch_size)

single_dataset_qpsk = MyDataset(qpsk_2_data_all, qpsk_labels)
single_loader_qpsk = DataLoader(single_dataset_qpsk, batch_size=batch_size)

for data, labels in single_dataset_bpsk:
    first_data_norm_bpsk = data
    first_data_label_norm_bpsk = labels
    break

for data, labels in single_dataset_qpsk:
    first_data_norm_qpsk = data
    first_data_label_norm_qpsk = labels
    break
print(f'bpsk norm {first_data_norm_bpsk}')
print(f'bpsk label {first_data_label_norm_bpsk}')

print(f'qpsk norm {first_data_norm_qpsk}')
print(f'qpsk label {first_data_label_norm_qpsk}')


with open('normalized_qpsk_first_sample.npz', 'wb') as file:
    np.savez(file, first_data_norm_bpsk.numpy(), first_data_norm_qpsk.numpy())
    #np.savez(file, first_data_norm_qpsk.numpy())


# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

bpsk norm tensor([[ 0.3931,  0.1058,  0.2165,  0.1036, -0.2535,  0.2177, -0.1325,  0.0406,
         -0.0508, -0.4125, -0.1632, -0.2335, -0.0145, -0.2577,  0.1194, -0.1942,
         -0.6206,  0.0697,  0.2044, -0.3668,  0.0674, -0.2717,  0.4861, -0.1844,
          0.2618, -0.1605,  0.2926,  0.1440,  0.2854,  0.5265,  0.1897,  0.2028,
          0.2990, -0.0814,  0.2161, -0.1558,  0.0926,  0.1711,  0.1701, -0.2396,
         -0.4849, -0.0088, -0.1630, -1.0000, -0.4429,  0.0955, -0.5022,  0.0499,
          0.4724,  0.1329,  0.2341,  0.2627,  0.1586,  0.0952,  0.0598,  0.1518,
          0.1579, -0.0184,  0.3549,  0.1710,  0.3485, -0.1053,  0.2578,  0.0291,
          0.2053,  1.0000, -0.0090,  0.2671, -0.1268, -0.1246,  0.3915, -0.1319,
          0.0191,  0.0581, -0.2850, -0.1924, -0.5802, -0.4047, -0.0812, -0.3177,
         -0.2312, -0.6280, -0.2524,  0.0702,  0.0185, -0.2254,  0.3336,  0.0816,
          0.1644,  0.3294,  0.2283,  0.1577,  0.0829, -0.4049, -0.3154, -0.1680,
          0.0349, 

C:\Users\calla\AppData\Local\Temp\ipykernel_19156\2117244470.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
C:\Users\calla\AppData\Local\Temp\ipykernel_19156\2117244470.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


In [6]:

import torch
import torch.nn as nn

class CNN2D(nn.Module):
    def __init__(self, num_classes):
        super(CNN2D, self).__init__()
        
        in_channels = 2
        # For the 2D conversion, we're assuming height=1 for the input
        height = 1
        width = 128  # Original length dimension
        
        # Upsample (adjusted for 2D)
        self.upsample = nn.Upsample(scale_factor=(1, 2), mode='bilinear', align_corners=False)  # height stays the same, width is doubled
        
        # Define 2D convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(1, 3), padding=(0, 1))
        self.relu1 = nn.ReLU()
        #self.maxpool1 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool1 = nn.AdaptiveAvgPool2d((1,64))
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 3), padding=(0, 1))
        self.relu2 = nn.ReLU()
        #self.maxpool2 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool2 = nn.AdaptiveAvgPool2d((1,32))
        
        # Adaptive pooling (2D)
        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        
        # Define the fully connected layers (adjusted for adaptive pooling output)
        self.fc1 = nn.Linear(128, 256)  # The input features after pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        # Adjust input shape for 2D processing
        x = x.unsqueeze(-2)  # Insert a height dimension of size 1
        
        # Upsample
        x = self.upsample(x)
        
        # 2D convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        #x = self.maxpool1(x)
        x = self.adaptive_pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        #x = self.maxpool2(x)
        x = self.adaptive_pool2(x)
        
        # Adaptive average pooling
        x = self.adaptive_avg_pool2d(x)
        
        # Flatten (before fully connected layers)
        x = x.reshape(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x


model = CNN2D(2)
print(model) #model info

CNN2D(
  (upsample): Upsample(scale_factor=(1.0, 2.0), mode='bilinear')
  (conv1): Conv2d(2, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu1): ReLU()
  (adaptive_pool1): AdaptiveAvgPool2d(output_size=(1, 64))
  (conv2): Conv2d(64, 128, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu2): ReLU()
  (adaptive_pool2): AdaptiveAvgPool2d(output_size=(1, 32))
  (adaptive_avg_pool2d): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


In [ ]:
# Specify a target name or fingerprint you want to deploy on
#target = "DPUCAHX8L_ISA0_SP"
target = "DPUCVDX8G_ISA3_C32B3"
#DPUCVDX8G
# Initialize inspector with target
inspector = Inspector(target)
# Note: visualization of inspection results relies on the dot engine.If you don't install dot successfully, set 'image_format = None' when inspecting.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN2D(2)

#dummy_data, labels = train_loader
#data_dummy = []
for data, labels in train_loader:
    data_dummy = data
    break
print(type(data_dummy))
print(data_dummy.shape)
dummy_input = torch.randn(1, 2, 128)
print(dummy_input)

inspector.inspect(model, (data_dummy.clone().detach().requires_grad_(True)), device=device, output_dir="inspect", image_format=None) 

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


learning_rate = 0.001
num_epochs = 3

# loss function and optimizer definition
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# training
for epoch in range(num_epochs):
    model.train()  # Set the model in training mode
    
    for data, labels in train_loader:
        #print(labels)
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    
    # print loss
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}')

C:\Users\calla\AppData\Local\Temp\ipykernel_19156\2117244470.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
C:\Users\calla\AppData\Local\Temp\ipykernel_19156\2117244470.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
C:\Users\calla\AppData\Local\Temp\ipykernel_19156\2117244470.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


Epoch [1/3] Loss: 0.0040
Epoch [2/3] Loss: 0.0014
Epoch [3/3] Loss: 0.1403


In [ ]:
def evaluate (model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            print(outputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # predicted and actual labels
            for i in range(len(labels)):
                print(f'Predicted: {predicted[i]}, Actual: {labels[i]}')
            accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy
#accuracy_orig_model = evaluate(model)

In [13]:
def evaluate_single (model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in single_loader_qpsk:
            print(data)
            outputs = model(data)
            print(outputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # predicted and actual labels
            for i in range(len(labels)):
                print(f'Predicted: {predicted[i]}, Actual: {labels[i]}')
                break
            accuracy = 100 * correct / total
            break
        print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy
accuracy_orig_model = evaluate_single(model)

tensor([[[-0.5538, -0.2223, -0.0730, -0.5020, -0.5433, -0.3482, -0.4678,
          -0.6157, -0.8473, -0.4545, -0.9437, -0.4267, -0.4059, -0.4823,
          -0.5700,  0.1071, -0.4377, -0.6519, -0.6253, -0.6040, -0.3602,
          -0.7856, -0.9484, -0.6470, -0.9013, -0.1484, -0.2725, -0.4248,
           0.3144,  0.6873,  0.3095,  0.7075,  0.8194,  0.3161,  0.1681,
           0.5946, -0.0188, -0.2125, -0.4877, -0.3429, -0.9482, -0.6003,
          -0.5690, -0.7183, -0.3251,  0.0377,  0.2358,  0.4388,  0.4312,
           0.6003,  0.7926,  0.4612,  0.0135,  0.0200, -0.2740, -0.5032,
          -0.8714, -0.4747, -0.4548,  0.0074, -0.0547,  0.0785,  0.6312,
           0.7444,  0.7589,  0.6345,  0.3086,  0.2407, -0.2376, -0.0604,
          -0.7498, -1.0000, -0.7669, -0.8912, -0.4371, -0.1114,  0.2022,
           0.4612,  0.1985,  0.5891,  0.9715,  0.6943,  0.5779,  0.2703,
           0.1050,  0.0180, -0.4597, -0.3135, -0.6197, -0.4616, -0.6835,
          -0.5644, -0.1236,  0.0358,  0.0210,  0.28

C:\Users\calla\AppData\Local\Temp\ipykernel_19156\2117244470.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
C:\Users\calla\AppData\Local\Temp\ipykernel_19156\2117244470.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


In [ ]:
print(total)

In [ ]:
torch.save(model, "trainedModel")

In [ ]:
from pytorch_nndct.apis import torch_quantizer, dump_xmodel



quantizer = torch_quantizer("calib", model, (dummy_input))
quant_model = quantizer.quant_model # is quantize aware

#accl_gen, acc4_gen, loss_get = evaluate(quant_model, val_loader, criterion)
# evaluate the model on test data
accuarcy_quant_model = evaluate(quant_model)

In [ ]:
quantizer.export_quant_config() # creates quantizer config.json

quantizer.export_torch_script #exports [model_name]_int.pt
quantizer.export_onnx_model() #exports [model_name]_int.onnx

quantizer.export_xmodel(deploy_check=True) # in therory exports xmodel, but exports [model_name].py
